In [36]:
import numpy as np
from grid import grid_world, Grid, print_values
import time

In [37]:
gamma = 0.9    # discount factor
theta = 0.1   # convergence threshold for policy evaluation

actions = ["up", "down", "left", "right"]

In [39]:
grid = grid_world()
print("Grid World:")
print(grid)

Grid World:
 S  .  .  . 
 .  .  .  . 
 .  .  .  . 
 .  .  .  G 



In [43]:
print_values(grid.rewards, grid)

-1.00 -1.00 -1.00 -1.00 
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.00 -1.00   G   


Setting random policy π

In [27]:
goal = (3, 3)  # goal state
policy = Grid.generate_random_policy(grid, goal)
print("Initial Random Policy:")
print(policy)


Initial Random Policy π:
 →  ←  ↓  ← 
 →  ↑  →  ← 
 →  ↓  →  ↑ 
 ↑  ↑  ←  G 
Initial Random Policy:
{(0, 0): 'right', (0, 1): 'left', (0, 2): 'down', (0, 3): 'left', (1, 0): 'right', (1, 1): 'up', (1, 2): 'right', (1, 3): 'left', (2, 0): 'right', (2, 1): 'down', (2, 2): 'right', (2, 3): 'up', (3, 0): 'up', (3, 1): 'up', (3, 2): 'left'}


In [28]:
grid.actions

{(0, 0): ['down', 'right'],
 (0, 1): ['down', 'left', 'right'],
 (0, 2): ['down', 'left', 'right'],
 (0, 3): ['down', 'left'],
 (1, 0): ['up', 'down', 'right'],
 (1, 1): ['up', 'down', 'left', 'right'],
 (1, 2): ['up', 'down', 'left', 'right'],
 (1, 3): ['up', 'down', 'left'],
 (2, 0): ['up', 'down', 'right'],
 (2, 1): ['up', 'down', 'left', 'right'],
 (2, 2): ['up', 'down', 'left', 'right'],
 (2, 3): ['up', 'down', 'left'],
 (3, 0): ['up', 'right'],
 (3, 1): ['up', 'left', 'right'],
 (3, 2): ['up', 'left', 'right']}

Policy Evaluation

In [ ]:
def policy_evaluation(grid, policy, gamma=0.9, theta=0.01):
    # value dict
    V = {state: 0.0 for state in grid.actions}
    # print("Initial Value Function:")
    # print_values(V, grid)
    converged = False
    while not converged:
        delta = 0
        for s in grid.actions:
            val = V[s]
            if grid.is_terminal(s):
                continue
            action = policy[s]
            next_state = grid.take_action(s, action)
            reward = grid.rewards[s]
            # P(s'|s,a) = 1, so we ignore it
            V[s] = reward + gamma * V.get(next_state, 0)
            delta = max(delta, abs(V[s] - val))
            # print_values(V, grid)
        
        if delta < theta:
            converged = True
    print("Value Function after Policy Evaluation:")
    print_values(V, grid)
    return V

    

V = policy_evaluation(grid, policy, gamma, theta)
print(V)
    
    


-1.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00 -2.71  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00 -2.71 -1.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00 -2.71 -1.00 -1.90 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00 -2.71 -1.00 -1.90 
-1.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.90 -1.00 -1.90 
-1.00 -2.71 -1.00 -1.90 
-1.00 -1.00  0.00  0.00 
 0.00  0.00  0.00   G   


Policy Improvement

In [31]:
def policy_improvement(grid, V, gamma):
    improved_policy = {}
    # print("\n--- Improving Policy ---")
    for s in grid.actions:
        if grid.is_terminal(s):
            continue
        best_action = None
        best_val = float("-inf")
        # print(f"\nEvaluating actions for state {s}:")
        for a in actions:
            next_state = grid.take_action(s, a)
            reward = grid.rewards[s]
            val = reward + gamma * V.get(next_state, 0)
            # print(f"  Action {a} → next state {next_state}, value = {val:.2f}")

            if val > best_val:
                best_val = val
                best_action = a
        improved_policy[s] = best_action
        # print(f"  → Best action: {best_action} with value {best_val:.2f}")


    print("\nImproved Policy:")
    for s in grid.actions:
        if s in improved_policy:
            print(f"  State {s}: {improved_policy[s]}")
    return improved_policy





In [32]:
new_policy = policy_improvement(grid, V, gamma)
print("New Policy:")
print(new_policy)
grid.print_policy(new_policy, grid)


Improved Policy:
  State (0, 0): up
  State (0, 1): right
  State (0, 2): up
  State (0, 3): left
  State (1, 0): down
  State (1, 1): down
  State (1, 2): up
  State (1, 3): up
  State (2, 0): left
  State (2, 1): left
  State (2, 2): up
  State (2, 3): down
  State (3, 0): up
  State (3, 1): up
  State (3, 2): right
New Policy:
{(0, 0): 'up', (0, 1): 'right', (0, 2): 'up', (0, 3): 'left', (1, 0): 'down', (1, 1): 'down', (1, 2): 'up', (1, 3): 'up', (2, 0): 'left', (2, 1): 'left', (2, 2): 'up', (2, 3): 'down', (3, 0): 'up', (3, 1): 'up', (3, 2): 'right'}
Policy π:
 ↑  →  ↑  ← 
 ↓  ↓  ↑  ↑ 
 ←  ←  ↑  ↓ 
 ↑  ↑  →  G 


Policy Iteration

In [33]:
def policy_iteration(grid, gamma=0.9, theta=0.01):
    policy = Grid.generate_random_policy(grid, grid.goal)

    while True:
        V = policy_evaluation(grid, policy, gamma, theta)
        new_policy = policy_improvement(grid, V, gamma)

        if new_policy == policy:
            break
        policy = new_policy

    return policy, V


In [35]:
print("\nRunning Full Policy Iteration...\n")
optimal_policy, optimal_values = policy_iteration(grid, gamma, theta)

print("\nFinal Optimal Policy:")
grid.print_policy(optimal_policy, grid)

print("\nFinal Value Function:")
print_values(optimal_values, grid)
print("Optimal Policy:")
grid.print_policy(optimal_policy, grid)



Running Full Policy Iteration...

Initial Random Policy π:
 →  ↓  ↓  ↓ 
 →  ←  ↑  ← 
 ↑  ↓  ↑  ↓ 
 →  ↑  ↑  G 
-1.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
-1.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.90  0.00  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.90 -1.90  0.00 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.90 -1.90 -2.71 
 0.00  0.00  0.00  0.00 
 0.00  0.00  0.00   G   
-1.00 -1.00 -1.00 -1.00 
-1.00 -1.90 -1.90 -2.71 
-1.90  0.00  0.00  0.00 
 0.00  0.00  